In [1]:
%cd ..

/volatile/home/Zaccharie/workspace/understanding-unets


In [2]:
# # this just to make sure we are using only on CPU
# import os
# os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [3]:
%load_ext autoreload
%autoreload 2
import os.path as op
import time

from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, LearningRateScheduler
from keras_tqdm import TQDMNotebookCallback
import tensorflow as tf
from tqdm import tqdm_notebook

from learning_wavelets.datasets import im_dataset_div2k, im_dataset_bsd500
from learning_wavelets.evaluate import psnr, ssim
from learning_wavelets.keras_utils.filters_cback import NormalizeWeights
from learning_wavelets.keras_utils.image_tboard_cback import TensorBoardImage
from learning_wavelets.keras_utils.normalisation import NormalisationAdjustment
from learning_wavelets.keras_utils.thresholding import DynamicSoftThresholding, DynamicHardThresholding
from learning_wavelets.learned_wavelet import learnlet

Using TensorFlow backend.


In [4]:
tf.random.set_seed(1)

In [5]:
noise_std_train = (0, 55)
noise_std_val = 30
batch_size = 8
source = 'bsd500'
if source == 'bsd500':
    data_func = im_dataset_bsd500
    n_samples_train = 400
else:
    data_func = im_dataset_div2k
    n_samples_train = 800
im_ds_train = data_func(
    mode='training', 
    batch_size=batch_size, 
    patch_size=128, 
    noise_std=noise_std_train, 
    return_noise_level=True,
)
im_ds_val = data_func(
    mode='validation', 
    batch_size=1, 
    patch_size=256, 
    noise_std=noise_std_val, 
    return_noise_level=True,
)

In [6]:
alpha = 2
run_params = {
    'denoising_activation': DynamicSoftThresholding(alpha, trainable=True),
    'learnlet_analysis_kwargs':{
        'n_tiling': 256, 
        'mixing_details': False,        
    },
    'learnlet_synthesis_kwargs': {
        'res': False,
    },
    'n_scales': 5,
    'exact_reconstruction_weight': 0,
    'clip': False,
}
n_epochs = 250
run_id = f'learnlet_dynamic_st_{source}_{noise_std_train[0]}_{noise_std_train[1]}_{int(time.time())}'
chkpt_path = f'checkpoints/{run_id}' + '-{epoch:02d}.hdf5'
print(run_id)

learnlet_dynamic_st_bsd500_0_55_1575975254


In [7]:
def l_rate_schedule(epoch):
    return max(1e-3 / 2**(epoch//25), 1e-5)
lrate_cback = LearningRateScheduler(l_rate_schedule)

In [8]:
chkpt_cback = ModelCheckpoint(chkpt_path, period=n_epochs, save_weights_only=False)
log_dir = op.join('logs', run_id)
tboard_cback = TensorBoard(
    log_dir=log_dir, 
    histogram_freq=0, 
    write_graph=True, 
    write_images=False, 
    profile_batch=0,
)
tqdm_cb = TQDMNotebookCallback(metric_format="{name}: {value:e}")
tqdm_cb.on_train_batch_begin = tqdm_cb.on_batch_begin
tqdm_cb.on_train_batch_end = tqdm_cb.on_batch_end
# tqdm_cb = TQDMProgressBar()
val_noisy, val_gt = next(iter(im_ds_val))
tboard_image_cback = TensorBoardImage(
    log_dir=log_dir + '/images',
    image=val_gt[0:1],
    noisy_image=val_noisy[0:1],
)
norm_cback = NormalisationAdjustment(momentum=0.99, n_pooling=5, dynamic_denoising=True)
norm_cback.on_train_batch_end = norm_cback.on_batch_end

W1210 11:54:14.233514 139653627713280 callbacks.py:863] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


In [9]:
n_channels = 1
model = learnlet(input_size=(None, None, n_channels), lr=1e-3, **run_params)
print(model.summary(line_length=114))

Model: "model"
__________________________________________________________________________________________________________________
Layer (type)                         Output Shape             Param #       Connected to                          
input_1 (InputLayer)                 [(None, None, None, 1)]  0                                                   
__________________________________________________________________________________________________________________
learnlet_analysis (LearnletAnalysis) [(None, None, None, 256) 32050         input_1[0][0]                         
__________________________________________________________________________________________________________________
normalisation (Normalisation)        (None, None, None, 256)  256           learnlet_analysis[0][0]               
                                                                            dynamic_soft_thresholding[0][0]       
_________________________________________________________________

In [ ]:
%%time
model.fit(
    im_ds_train, 
    steps_per_epoch=int(n_samples_train / batch_size), 
#     steps_per_epoch=5, 
    epochs=n_epochs,
    validation_data=im_ds_val,
#     validation_steps=int(validation_split * n_samples_train / batch_size),
    validation_steps=1,
    verbose=0,
    callbacks=[tqdm_cb, tboard_cback, chkpt_cback, norm_cback, lrate_cback],
#     callbacks=[tqdm_cb, tboard_cback, chkpt_cback, tboard_image_cback, norm_cback, lrate_cback],
    shuffle=False,
)

W1210 11:54:20.453922 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.237121). Check your callbacks.
W1210 11:54:20.681545 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.145692). Check your callbacks.
W1210 11:54:20.897024 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.145357). Check your callbacks.
W1210 11:54:21.113930 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.145023). Check your callbacks.
W1210 11:54:21.328769 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.139840). Check your callbacks.
W1210 11:54:21.552300 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.138362). Check your callbacks.
W1210 11:54:21.753277 139653627713280 callbacks.py:244] Method (on_tra

W1210 11:54:31.898234 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.132628). Check your callbacks.
W1210 11:54:32.115083 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.133491). Check your callbacks.
W1210 11:54:32.321343 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.132628). Check your callbacks.
W1210 11:54:32.544047 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.133491). Check your callbacks.
W1210 11:54:32.764588 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.134354). Check your callbacks.
W1210 11:54:32.985233 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.134790). Check your callbacks.
W1210 11:54:33.212567 139653627713280 callbacks.py:244] Method (on_tra

W1210 11:54:43.370422 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.148885). Check your callbacks.
W1210 11:54:43.619667 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.154640). Check your callbacks.
W1210 11:54:43.841797 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.148885). Check your callbacks.
W1210 11:54:44.060523 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.145127). Check your callbacks.
W1210 11:54:44.287828 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.141370). Check your callbacks.
W1210 11:54:44.507081 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.140511). Check your callbacks.
W1210 11:54:44.742684 139653627713280 callbacks.py:244] Method (on_tra

W1210 11:54:55.570897 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.159821). Check your callbacks.
W1210 11:54:55.820296 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.163554). Check your callbacks.
W1210 11:54:56.074495 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.167287). Check your callbacks.
W1210 11:54:56.296054 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.163554). Check your callbacks.
W1210 11:54:56.510184 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.159821). Check your callbacks.
W1210 11:54:56.727063 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.148636). Check your callbacks.
W1210 11:54:56.937864 139653627713280 callbacks.py:244] Method (on_tra

W1210 11:55:06.767887 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.152300). Check your callbacks.
W1210 11:55:06.984232 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.142888). Check your callbacks.
W1210 11:55:07.225026 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.152300). Check your callbacks.
W1210 11:55:07.494383 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.156111). Check your callbacks.
W1210 11:55:07.716886 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.152300). Check your callbacks.
W1210 11:55:07.927504 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.145853). Check your callbacks.
W1210 11:55:08.132111 139653627713280 callbacks.py:244] Method (on_tra

...

W1210 11:59:00.887349 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.137160). Check your callbacks.
W1210 11:59:01.104929 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.136540). Check your callbacks.
W1210 11:59:01.319913 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.135919). Check your callbacks.
W1210 11:59:01.536162 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.134744). Check your callbacks.
W1210 11:59:01.757403 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.135919). Check your callbacks.
W1210 11:59:01.967649 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.134744). Check your callbacks.
W1210 11:59:02.215165 139653627713280 callbacks.py:244] Method (on_tra

W1210 11:59:12.346081 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.132906). Check your callbacks.
W1210 11:59:12.568761 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.135439). Check your callbacks.
W1210 11:59:12.804079 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.137971). Check your callbacks.
W1210 11:59:13.061517 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.145950). Check your callbacks.
W1210 11:59:13.297278 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.153928). Check your callbacks.
W1210 11:59:13.538635 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.154283). Check your callbacks.
W1210 11:59:13.787600 139653627713280 callbacks.py:244] Method (on_tra

W1210 11:59:24.448839 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.133462). Check your callbacks.
W1210 11:59:24.664728 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.133505). Check your callbacks.
W1210 11:59:24.881191 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.133548). Check your callbacks.
W1210 11:59:25.112060 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.133619). Check your callbacks.
W1210 11:59:25.331670 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.133689). Check your callbacks.
W1210 11:59:25.557587 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.135481). Check your callbacks.
W1210 11:59:25.803642 139653627713280 callbacks.py:244] Method (on_tra

W1210 11:59:35.851656 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.135193). Check your callbacks.
W1210 11:59:36.067769 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.134814). Check your callbacks.
W1210 11:59:36.308948 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.135193). Check your callbacks.
W1210 11:59:36.559956 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.147738). Check your callbacks.
W1210 11:59:36.805004 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.160283). Check your callbacks.
W1210 11:59:37.044101 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.158903). Check your callbacks.
W1210 11:59:37.283173 139653627713280 callbacks.py:244] Method (on_tra

W1210 11:59:48.177110 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.133700). Check your callbacks.
W1210 11:59:48.410446 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.143070). Check your callbacks.
W1210 11:59:48.641068 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.150028). Check your callbacks.
W1210 11:59:48.878803 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.151234). Check your callbacks.
W1210 11:59:49.119999 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.152440). Check your callbacks.
W1210 11:59:49.360140 139653627713280 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.153357). Check your callbacks.
W1210 11:59:49.618755 139653627713280 callbacks.py:244] Method (on_tra

In [ ]:
# %%time
# # overfitting trials
# data = next(iter(im_ds_train))
# val_data = next(iter(im_ds_val))
# model.fit(
#     x=data[0], 
#     y=data[1], 
# #     validation_data=val_data, 
#     batch_size=1, 
# #     callbacks=[tqdm_cb, tboard_cback, tboard_image_cback, norm_cback, lrate_cback],
#     callbacks=[tqdm_cb, tboard_cback, norm_cback,],
#     epochs=50, 
#     verbose=2, 
#     shuffle=False,
# )
# # print('Original metrics')
# # print(psnr(data[0].numpy(), data[1].numpy()))
# # print(ssim(data[0].numpy(), data[1].numpy()))

In [ ]:
dyn = [l for l in model.layers if 'dynamic' in l.name]

In [ ]:
[l.alpha for l in dyn]